In [1]:
import numpy as np;
import pandas as pd;
import re;
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/harish/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [4]:
news_df = pd.read_csv("./news_data.csv");

In [5]:
news_df.shape

(20800, 5)

In [6]:
news_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
news_df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
news_df.fillna('',inplace=True);

In [9]:
news_df['content'] = news_df['author']+ " " + news_df['title']

In [10]:
stemmer = PorterStemmer();

def convertText(content):
    stemmed = re.sub('[^a-zA-Z]',' ',content)
    stemmed = stemmed.lower()
    stemmed = stemmed.split()
    stemmed = [stemmer.stem(word) for word in stemmed if not word in stopwords.words('english')]
    stemmed = ' '.join(stemmed)
    return stemmed

In [11]:
news_df['content'] = news_df['content'].apply(convertText)

In [12]:
x = news_df['content'].values;
y = news_df['label'].values;

In [13]:
print(x[:5])

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire'
 'jessica purkiss civilian kill singl us airstrik identifi'
 'howard portnoy iranian woman jail fiction unpublish stori woman stone death adulteri']


In [14]:
print(y[:10])

[1 0 1 1 1 0 1 0 0 0]


In [15]:
vectorizer = TfidfVectorizer();
vectorizer.fit(x);
x = vectorizer.transform(x);

In [16]:
print(x.shape)
print(x[0])

(20800, 17128)
  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766


In [17]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=2)

In [18]:
print(x_train.shape)
print(x_test.shape)

(16640, 17128)
(4160, 17128)


In [19]:
model = LogisticRegression();


In [20]:
model.fit(x_train,y_train)

LogisticRegression()

In [21]:
y_predict_train = model.predict(x_train)
print(f"Accuracy on training data : {accuracy_score(y_train,y_predict_train)}")

Accuracy on training data : 0.9873798076923077


In [22]:
y_predict_test = model.predict(x_test)
print(f"Accuracy on testing data : {accuracy_score(y_test,y_predict_test)}")

Accuracy on testing data : 0.9774038461538461


In [23]:
import joblib;

In [24]:
joblib.dump(model,'news_model.joblib')
joblib.dump(vectorizer,'tfidf_vectorizer.joblib');